In [58]:
# Let's do our usual data science imports
import pandas as pd
import numpy as np
import matplotlib as plot
import sklearn as sk
import importlib
import os
import subprocess


# Define a number of constants
STANDARD_MUTATION = 0.5
STANDARD_MOMENTUM = 0
STANDARD_POPSIZE = 100
STANDARD_SIGMA = 0.004
PATH_TO_LOG: "storage/logs/"
    
EVALUATION_SCHAFFERS = "SchaffersEvaluation"
EVALUATION_KATSUURA = "KatsuuraEvaluation"
EVALUATION_BCF = "BentCigarFunction"
EVALUATION_SPHERE = "SphereEvaluation"

In [2]:
# Define the Setup class
class Setup:
    """A setup for a defined parameter and evaluation, with given parameter values.

    Params:
        - param_name (str): Parameter to define
        - evaluation (str): Name of evaluation function to optimise.
        - parameter_values (list): List of values for the associated parameter
    """
    def __init__(self, param_name = "", evaluation = "", parameter_values = []):
        self.param_name = param_name
        self.evaluation = evaluation
        self.parameter_values = parameter_values
        self.parameter_scores = []

In [59]:
# Define the Simulator class
class Simulator:
    """A simulator for a givenn number of setups, and runs to test on each parameter value of a setup.

    Params:
        - setups(list): A list of Setup to test on
        - runs(int): Number of runs per setup
    """
    
    def __init__(self, setups, runs=10):
        self.setups = setups
        self.runs = runs

    def run(self):
        for setup in self.setups:
            self.runEvalsForSetup(setup)

    def constructParams(self, setup, param):
        """Sets up default and preconfigured parameters."""
        mutation = STANDARD_MUTATION
        momentum = STANDARD_MOMENTUM
        sigma = STANDARD_SIGMA
        popsize = STANDARD_POPSIZE

        if setup.name == "Mutation":
            mutation = param

        if setup.name == "Momentum":
            momentum = param

        if setup.name == "Sigma":
            sigma = param
        
        if setup.name == "Population Size":
            popsize = param

        return {
            'mutation': mutation,
            'momentum': momentum,
            'sigma': sigma,
            'popsize': popsize
        }

    def runEvalsForSetup(self, setup):
        """Runs for the current Setup:setup of parameters to test. """
        for param in setup.parameter_values:
            params = self.constructParams(setup, param)
            self.runEval(setup, params)

    def runEval(self, setup_params ,params):
        """Runs the EA on the igiven eval"""
        scores = 0
        
        for i in range(self.runs):
            cmd = "bash run.sh %s %d %f" % (setup.evaluation, params['popsize'], params['sigma'])
            p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
            output = p.stdout.readlines()
            scores += output
            retval = p.wait()

In [ ]:
cmd = "bash run.sh SchaffersEvaluation 20 0.004"
p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
output = p.stdout.readlines()

In [57]:
print(output)

[b'[9.58803341365837, 9.541408688759521, 9.637311626507966, 9.542520087897095, 9.568733080167236, 9.543467352746097, 9.586664337667944, 9.768324802966559, 9.627250258634582, 9.485936983165447, 9.621948411720396, 9.58010680909793, 9.66515739461559, 9.419645843901767, 9.622162537247691, 9.703562368785564, 9.583781082077863, 9.58879232403943, 9.62346914841321, 9.140980248470141, 9.582126636564364, 9.697710573851827, 9.722019268085894, 9.675414736262397, 9.687361726552817, 9.493761734227277, 9.671670090742554, 9.604667593798531, 9.69076289924954, 9.583704232705585, 9.731169986755734, 9.746282746532291, 9.557999969682372, 9.55443004544102, 9.553783945536344, 9.510305871904569, 9.489701948737267, 9.601641178473475, 9.661951683963068, 9.753106679303977, 9.751670699820249, 9.497217159533992, 9.665375494538456, 9.727279787860116, 9.766870573320038, 9.799283426868065, 9.586404393061953, 9.501890730093411, 9.824167863650047, 9.593384469876913, 9.581131827159892, 9.593840459061235, 9.8136340371128